# IUB Peak Demand - Training Model

In [105]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pickle import dump
from pickle import load

In [106]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

## Import imputed meter data integrated with weather data

In [107]:
# Read in the data, it is tab seperated.  Note: Google Co-Lab has trouble reading the file, likely due to some type of encoding issue.
relativePath=os.getcwd()
# readFilePath=relativePath+"\\IU_Peak.txt"
readFilePath=relativePath+"\\iu_imputed_with_weather.txt"
df = pd.read_csv(readFilePath, sep="\t")

In [108]:
df.head()

Unnamed: 0     READ_DT  READ_HR  READ_30MIN_INT  READ_15MIN_INT  \
0           0  2019-01-01        1               1               1   
1           1  2019-01-01        1               1               2   
2           2  2019-01-01        1               2               3   
3           3  2019-01-01        1               2               4   
4           4  2019-01-01        2               1               1   

              READ_DTM   READ_VALUE  METER_4440104_MAIN  \
0  2019-01-01 00:15:00  4608.560423         1059.262573   
1  2019-01-01 00:30:00  4601.997580         1056.900635   
2  2019-01-01 00:45:00  4606.876556         1060.344604   
3  2019-01-01 01:00:00  4612.905275         1062.649292   
4  2019-01-01 01:15:00  4584.361818         1056.293335   

   METER_4440012_MAIN_CHILLER  METER_4440073_MAIN_CHILLER  ...        DATE  \
0                   17.532255                         0.0  ...  2019-01-01   
1                   17.296881                         0.0  ...  2019-01-01   
2                   17.307053                         0.0  ...  2019-01-01   
3                   16.875765                         0.0  ...  2019-01-01   
4                   17.346422                         0.0  ...  2019-01-01   

   WEEKDAY            date_time  temperature  wind_chill  precipitation  \
0        1  2019-01-01 00:00:00         44.9        37.8            NaN   
1        1  2019-01-01 00:00:00         44.9        37.8            NaN   
2        1  2019-01-01 00:00:00         44.9        37.8            NaN   
3        1  2019-01-01 01:00:00         44.0        37.3            NaN   
4        1  2019-01-01 01:00:00         44.0        37.3            NaN   

   wind_speed  relative_humidity  conditions academic_period  
0        16.1              79.47    Overcast             off  
1        16.1              79.47    Overcast             off  
2        16.1              79.47    Overcast             off  
3        13.9              82.24    Overcast             off  
4        13.9              82.24    Overcast             off  

[5 rows x 29 columns]

In [109]:
df.shape

(35140, 29)

In [110]:
df.columns

Index(['Unnamed: 0', 'READ_DT', 'READ_HR', 'READ_30MIN_INT', 'READ_15MIN_INT',
       'READ_DTM', 'READ_VALUE', 'METER_4440104_MAIN',
       'METER_4440012_MAIN_CHILLER', 'METER_4440073_MAIN_CHILLER',
       'METER_4440177_MAIN', 'METER_4440105_MAIN', 'METER_4440099_MAIN',
       'METER_4440145_CHILLER', 'METER_4440071_CHILLER',
       'METER_4440134_CHILLER', 'METER_4440106_CHILLER', 'MONTH', 'HOUR',
       'DATE', 'WEEKDAY', 'date_time', 'temperature', 'wind_chill',
       'precipitation', 'wind_speed', 'relative_humidity', 'conditions',
       'academic_period'],
      dtype='object')

In [111]:
main_cols = ['READ_DT', 'READ_HR',  'READ_30MIN_INT',  'READ_15MIN_INT',  'READ_DTM']  

In [112]:
we_cols = ['temperature', 'precipitation', 'wind_speed', 'relative_humidity', 'conditions']

In [113]:
read_cols = ['READ_VALUE', 'METER_4440104_MAIN',  'METER_4440012_MAIN_CHILLER',  'METER_4440073_MAIN_CHILLER',
           'METER_4440177_MAIN', 'METER_4440105_MAIN', 'METER_4440099_MAIN']

In [114]:
df_main = df.loc[:, main_cols + we_cols + read_cols]
df_main.head()

READ_DT  READ_HR  READ_30MIN_INT  READ_15MIN_INT             READ_DTM  \
0  2019-01-01        1               1               1  2019-01-01 00:15:00   
1  2019-01-01        1               1               2  2019-01-01 00:30:00   
2  2019-01-01        1               2               3  2019-01-01 00:45:00   
3  2019-01-01        1               2               4  2019-01-01 01:00:00   
4  2019-01-01        2               1               1  2019-01-01 01:15:00   

   temperature  precipitation  wind_speed  relative_humidity conditions  \
0         44.9            NaN        16.1              79.47   Overcast   
1         44.9            NaN        16.1              79.47   Overcast   
2         44.9            NaN        16.1              79.47   Overcast   
3         44.0            NaN        13.9              82.24   Overcast   
4         44.0            NaN        13.9              82.24   Overcast   

    READ_VALUE  METER_4440104_MAIN  METER_4440012_MAIN_CHILLER  \
0  4608.560423         1059.262573                   17.532255   
1  4601.997580         1056.900635                   17.296881   
2  4606.876556         1060.344604                   17.307053   
3  4612.905275         1062.649292                   16.875765   
4  4584.361818         1056.293335                   17.346422   

   METER_4440073_MAIN_CHILLER  METER_4440177_MAIN  METER_4440105_MAIN  \
0                         0.0          246.069977          390.834778   
1                         0.0          241.719467          397.691437   
2                         0.0          241.548355          402.332794   
3                         0.0          233.895752          404.062347   
4                         0.0          233.817581          388.064148   

   METER_4440099_MAIN  
0         2894.860840  
1         2888.389160  
2         2885.343750  
3         2895.422119  
4         2888.840332

In [115]:
df_main.shape

(35140, 17)

In [116]:
df_main.columns

Index(['READ_DT', 'READ_HR', 'READ_30MIN_INT', 'READ_15MIN_INT', 'READ_DTM',
       'temperature', 'precipitation', 'wind_speed', 'relative_humidity',
       'conditions', 'READ_VALUE', 'METER_4440104_MAIN',
       'METER_4440012_MAIN_CHILLER', 'METER_4440073_MAIN_CHILLER',
       'METER_4440177_MAIN', 'METER_4440105_MAIN', 'METER_4440099_MAIN'],
      dtype='object')

## Feature Engineering

### DATE split

In [117]:
df_main.READ_DT = pd.to_datetime(df.READ_DT)
df_main.READ_DTM = pd.to_datetime(df.READ_DTM)

df_main['MONTH'] = df_main['READ_DTM'].dt.month
df_main['HOUR'] = df_main['READ_DTM'].dt.hour
df_main['DATE'] = df_main['READ_DTM'].dt.date
df_main['WEEKDAY'] = df_main['READ_DTM'].dt.weekday_name
df_main['DAY'] = df_main['READ_DTM'].dt.day

### Academic_Period

In [118]:
df_main['academic_period'] = 'Break'
df_main['academic_period'] = np.where((df_main.READ_DT >= '2019-01-07') & (df_main.READ_DT <= '2019-05-03'),'Spring_AP',df_main.academic_period)
df_main['academic_period'] = np.where((df_main.READ_DT >= '2019-05-07') & (df_main.READ_DT <= '2019-07-26'),'Summer_AP',df_main.academic_period)
df_main['academic_period'] = np.where((df_main.READ_DT >= '2019-08-26') & (df_main.READ_DT <= '2019-12-20'),'Fall_AP',df_main.academic_period)

### Season

In [119]:
df_main['SEASON'] = ''
df_main['SEASON'] = np.where((df_main.READ_DT >= '2019-03-20') & (df_main.READ_DT <= '2019-06-20'),'Spring',df_main.SEASON)
df_main['SEASON'] = np.where((df_main.READ_DT >= '2019-06-21') & (df_main.READ_DT <= '2019-09-22'),'Summer',df_main.SEASON)
df_main['SEASON'] = np.where((df_main.READ_DT >= '2019-09-23') & (df_main.READ_DT <= '2019-12-20'),'Fall',df_main.SEASON)
df_main['SEASON'] = np.where((df_main.READ_DT >= '2019-12-21') & (df_main.READ_DT <= '2019-12-31'),'Winter',df_main.SEASON)
df_main['SEASON'] = np.where((df_main.READ_DT >= '2019-01-01') & (df_main.READ_DT <= '2019-03-19'),'Winter',df_main.SEASON)

### HOLIDAY Flag - Set to 1 for Weekends, Official Holidays, Recess period and for academic period breaks

In [120]:
HOLIDAYS = ['2019-01-20', '2019-05-27', '2019-07-04', '2019-09-02', 
            '2019-03-18', '2019-03-19', '2019-03-20', '2019-03-21', '2019-03-22', 
            '2019-11-25', '2019-11-26', '2019-11-27', '2019-11-28', '2019-11-29']

In [121]:
df_main['HOLIDAY'] = 0
df_main['HOLIDAY'] = np.where((df_main.READ_DT.isin(HOLIDAYS)) | (df_main.academic_period == 'Break')
                                 | (df_main.WEEKDAY.isin(['Saturday', 'Sunday'])), 1, df_main.HOLIDAY)

In [122]:
df_main.head()

READ_DT  READ_HR  READ_30MIN_INT  READ_15MIN_INT            READ_DTM  \
0 2019-01-01        1               1               1 2019-01-01 00:15:00   
1 2019-01-01        1               1               2 2019-01-01 00:30:00   
2 2019-01-01        1               2               3 2019-01-01 00:45:00   
3 2019-01-01        1               2               4 2019-01-01 01:00:00   
4 2019-01-01        2               1               1 2019-01-01 01:15:00   

   temperature  precipitation  wind_speed  relative_humidity conditions  ...  \
0         44.9            NaN        16.1              79.47   Overcast  ...   
1         44.9            NaN        16.1              79.47   Overcast  ...   
2         44.9            NaN        16.1              79.47   Overcast  ...   
3         44.0            NaN        13.9              82.24   Overcast  ...   
4         44.0            NaN        13.9              82.24   Overcast  ...   

   METER_4440105_MAIN  METER_4440099_MAIN  MONTH  HOUR        DATE  WEEKDAY  \
0          390.834778         2894.860840      1     0  2019-01-01  Tuesday   
1          397.691437         2888.389160      1     0  2019-01-01  Tuesday   
2          402.332794         2885.343750      1     0  2019-01-01  Tuesday   
3          404.062347         2895.422119      1     1  2019-01-01  Tuesday   
4          388.064148         2888.840332      1     1  2019-01-01  Tuesday   

   DAY  academic_period  SEASON HOLIDAY  
0    1            Break  Winter       1  
1    1            Break  Winter       1  
2    1            Break  Winter       1  
3    1            Break  Winter       1  
4    1            Break  Winter       1  

[5 rows x 25 columns]

In [123]:
df_main.shape

(35140, 25)

In [124]:
df_main.isnull().sum()

READ_DT                           0
READ_HR                           0
READ_30MIN_INT                    0
READ_15MIN_INT                    0
READ_DTM                          0
temperature                     201
precipitation                 34020
wind_speed                     7553
relative_humidity              1337
conditions                      193
READ_VALUE                        0
METER_4440104_MAIN                0
METER_4440012_MAIN_CHILLER        0
METER_4440073_MAIN_CHILLER        0
METER_4440177_MAIN                0
METER_4440105_MAIN                0
METER_4440099_MAIN                0
MONTH                             0
HOUR                              0
DATE                              0
WEEKDAY                           0
DAY                               0
academic_period                   0
SEASON                            0
HOLIDAY                           0
dtype: int64

### Imputing Missing values

In [125]:
# Drop Records in 2020 (i.e. SEASON = Blank)
df_main = df_main[df_main['SEASON'] != ''].reset_index(drop=True)

In [126]:
# df_main['wind_chill'].fillna(0, inplace=True)
df_main['precipitation'].fillna(0, inplace=True)
df_main['relative_humidity'].fillna(0, inplace=True)
df_main['wind_speed'].fillna(0, inplace=True)
# df_merge['conditions'].fillna('Clear', inplace=True)
# df_merge['temperature'].fillna(df_merge['temperature'].mean(), inplace=True)

## Create 6 dataframes 
* 1 each for a main meter

In [127]:
FE_cols = ['MONTH', 'HOUR', 'DAY', 'WEEKDAY', 'academic_period', 'SEASON', 'HOLIDAY']

In [128]:
df_duke_099 = df_main.loc[:, FE_cols + we_cols + ['METER_4440099_MAIN']].rename(columns = {'METER_4440099_MAIN' : 'READ_VALUE'})
df_duke_104 = df_main.loc[:, FE_cols + we_cols + ['METER_4440104_MAIN']].rename(columns = {'METER_4440104_MAIN' : 'READ_VALUE'})
df_duke_105 = df_main.loc[:, FE_cols + we_cols + ['METER_4440105_MAIN']].rename(columns = {'METER_4440105_MAIN' : 'READ_VALUE'})
df_duke_177 = df_main.loc[:, FE_cols + we_cols + ['METER_4440177_MAIN']].rename(columns = {'METER_4440177_MAIN' : 'READ_VALUE'})
df_duke_012 = df_main.loc[:, FE_cols + we_cols + ['METER_4440012_MAIN_CHILLER']].rename(columns = {'METER_4440012_MAIN_CHILLER' : 'READ_VALUE'})
df_duke_073 = df_main.loc[:, FE_cols + we_cols + ['METER_4440073_MAIN_CHILLER']].rename(columns = {'METER_4440073_MAIN_CHILLER' : 'READ_VALUE'})
# df_duke_AGG = df_main.loc[:, FE_cols + we_cols + ['READ_VALUE']]

In [129]:
df_duke_177.head()

MONTH  HOUR  DAY  WEEKDAY academic_period  SEASON  HOLIDAY  temperature  \
0      1     0    1  Tuesday           Break  Winter        1         44.9   
1      1     0    1  Tuesday           Break  Winter        1         44.9   
2      1     0    1  Tuesday           Break  Winter        1         44.9   
3      1     1    1  Tuesday           Break  Winter        1         44.0   
4      1     1    1  Tuesday           Break  Winter        1         44.0   

   precipitation  wind_speed  relative_humidity conditions  READ_VALUE  
0            0.0        16.1              79.47   Overcast  246.069977  
1            0.0        16.1              79.47   Overcast  241.719467  
2            0.0        16.1              79.47   Overcast  241.548355  
3            0.0        13.9              82.24   Overcast  233.895752  
4            0.0        13.9              82.24   Overcast  233.817581

In [130]:
df_duke_099.shape

(35044, 13)

In [131]:
df_duke_104.describe()

MONTH          HOUR           DAY       HOLIDAY   temperature  \
count  35044.000000  35044.000000  35044.000000  35044.000000  34939.000000   
mean       6.525625     11.499030     15.719895      0.408344     54.416941   
std        3.447910      6.922486      8.796083      0.491534     19.526592   
min        1.000000      0.000000      1.000000      0.000000     -7.700000   
25%        4.000000      5.000000      8.000000      0.000000     38.200000   
50%        7.000000     11.000000     16.000000      0.000000     55.900000   
75%       10.000000     17.000000     23.000000      1.000000     70.900000   
max       12.000000     23.000000     31.000000      1.000000     92.800000   

       precipitation    wind_speed  relative_humidity    READ_VALUE  
count   35044.000000  35044.000000       35044.000000  35044.000000  
mean        0.005125      6.253130          72.760103    944.382670  
std         0.037194      4.797522          22.243547    173.843119  
min         0.000000      0.000000           0.000000      0.000000  
25%         0.000000      3.400000          60.862500    807.735000  
50%         0.000000      5.800000          77.770000    930.035000  
75%         0.000000      9.200000          89.610000   1031.940000  
max         1.300000     40.300000         100.000000   1548.012939

## Modeling

### Prepare Datatsets

In [132]:
def prep_data(df_merge):
    X_train = df_merge.drop(['READ_VALUE'], axis=1)
    y_train = df_merge["READ_VALUE"]
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42, stratify=X['MONTH'])
#     X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
    
    # Get Feature Names
    num_features = X_train.select_dtypes(exclude=['object']).columns.tolist()
    cat_features = X_train.select_dtypes('object').columns.tolist()
    
    return X_train, y_train, num_features, cat_features

### Build Pipeline

In [133]:
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [134]:
def build_pipeline(X_train, num_features, cat_features):
    # Identify the numeric features we wish to consider. 
    num_attribs = num_features

    # Numerical Features mean imputing
    num_pipeline = Pipeline([
            ('selector', DataFrameSelector(num_attribs)),
            ('imputer', SimpleImputer(strategy='mean')),
            ('std_scaler', StandardScaler()),
        ])

    # Identify the categorical features we wish to consider.
    cat_attribs = cat_features

    cat_pipeline = Pipeline([
            ('selector', DataFrameSelector(cat_attribs)),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('ohe', OneHotEncoder(sparse=False, handle_unknown="ignore"))
        ])

    data_prep_pipeline = FeatureUnion(transformer_list=[
            ("num_pipeline", num_pipeline),
            ("cat_pipeline", cat_pipeline),
        ], n_jobs=4)
    
    # Fit and Transform Training Data
    X_train_trans = data_prep_pipeline.fit_transform(X_train)
    
    return X_train_trans, data_prep_pipeline

### Model - Random Forest Regressor

In [135]:
def ml_model(X_train_trans, y_train):
    rf = RandomForestRegressor(n_estimators = 300,
                               min_samples_split = 2,
                               min_samples_leaf = 1, 
                               max_features = 'auto',
                               max_depth = 30,
                               n_jobs = -1,
                               oob_score = True,
                               bootstrap = True,
                               random_state = 42)

    rf.fit(X_train_trans, y_train)
        
#     y_test_pred = rf.predict(X_test_trans)
    
#     rf_mse = mean_squared_error(y_test, y_test_pred)
#     rf_rmse = np.sqrt(rf_mse)
    
    return rf

## Final Training Models

In [136]:
df_list = [df_duke_099, df_duke_104, df_duke_105, df_duke_177, df_duke_012, df_duke_073]

In [137]:
ID = ['4440099', '4440104', '4440105', '4440177', '4440012', '4440073']

In [138]:
%%time
def main():
    
    for c, df_duke in enumerate(df_list):
        X_train, y_train, num_features, cat_features = prep_data(df_duke)
        print(X_train.shape, y_train.shape)

        X_train_trans, data_prep_pipeline = build_pipeline(X_train, num_features, cat_features)
        print(X_train_trans.shape)

        pipe_name = 'pipeline_' + ID[c] + '.pkl'
        dump(data_prep_pipeline, open(pipe_name, 'wb'))
        
        rf = ml_model(X_train_trans, y_train)
        
        model_name = 'model_' + ID[c] + '.pkl'
        dump(rf, open(model_name, 'wb'))
        
if __name__ == "__main__":
    main()

(35044, 12) (35044,)
(35044, 29)
(35044, 12) (35044,)
(35044, 29)
(35044, 12) (35044,)
(35044, 29)
(35044, 12) (35044,)
(35044, 29)
(35044, 12) (35044,)
(35044, 29)
(35044, 12) (35044,)
(35044, 29)
Wall time: 2min 57s
